In [2]:
!../data_processing/setup_spacy.sh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 5.2 MB/s eta 0:00:00m eta 0:00:010:01:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
import sys; sys.path.append('../data_processing/')
import pandas as pd
from parse_dep import *
import re
from ast import literal_eval
import tokenizations
from datasets import load_dataset

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [4]:
bpe_pipeline = get_spacy_pipeline()

In [5]:
df = pd.read_csv('wiki_text_test_parsed2.pred.mwe', sep='\t', names=[0,'sentence','d'],)

In [6]:
wiki_text = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")['text']
wiki_text = [w for w in wiki_text if w]

Found cached dataset wikitext (/Users/andrejerkelens/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


In [8]:
df['d'] = df.apply(lambda x: literal_eval(x['d']), axis=1)

In [9]:
df = pd.concat([df, pd.json_normalize(df.d)],axis=1)
df = df.drop(columns=[0])

In [10]:
df['tokens'] = df['toks'].apply(lambda x: [y[0] for y in x])
df['sentence'] = df['sentence'].apply(lambda x: literal_eval(x).decode('utf-8'))

In [11]:
df.sentence[0]

'= Robert_Boulter ='

In [31]:
df.shape

(10265, 8)

---

### Note:

Only piece is to remove underscore from sentences 

In [32]:
nlp = spacy.load("en_core_web_sm")

In [33]:
def parse_sentences_mwe(dataset, pipeline = "spacy"):
    lines = []
    for d in dataset:
        if pipeline == "spacy":
            doc = nlp(d)
        elif pipeline == "bpe":
            doc = bpe_pipeline(d)
        if doc:
            for sent in doc.sents:
                lines.append(f"{word}\t{word.pos_}\n")
                lines.append("\n")
    return lines

In [16]:
df.iloc[1].tokens

['Robert',
 'Boulter',
 'is',
 'an',
 'English',
 'film',
 ',',
 'television',
 'and',
 'theatre',
 'actor',
 '.']

In [25]:
list(map(str, list(bpe_pipeline(wiki_text[1]).sents)[0]))

['ĠRobert',
 'ĠB',
 'oul',
 'ter',
 'Ġis',
 'Ġan',
 'ĠEnglish',
 'Ġfilm',
 'Ġ,',
 'Ġtelevision',
 'Ġand',
 'Ġtheatre',
 'Ġactor',
 'Ġ.',
 'ĠHe',
 'Ġhad',
 'Ġa',
 'Ġguest',
 'Ġ@',
 '-',
 '@']

In [29]:
_, two2one = tokenizations.get_alignments(df.iloc[1].tokens, list(map(str, list(bpe_pipeline(wiki_text[1]).sents)[0])))

In [30]:
two2one

[[0],
 [1],
 [1],
 [1],
 [2],
 [3],
 [4],
 [5],
 [6],
 [7],
 [8],
 [9],
 [10],
 [11],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [35]:
df['sent'] = df['sentence'].apply(lambda x: x.replace("_", " ").replace("~", " "))

In [37]:
df

,sentence,d,toks,tags,_,~,lemmas,tokens,sent
0,= Robert_Boulter =,"{'toks': [['=', 'VERB'], ['Robert', 'PROPN'], ...","[[=, VERB], [Robert, PROPN], [Boulter, PROPN],...","[O, B-GROUP, Ī, O]","[[2, 3]]",[],"[=, robert, boulter, =]","[=, Robert, Boulter, =]",= Robert Boulter =
1,"Robert_Boulter is an English film , television...","{'toks': [['Robert', 'PROPN'], ['Boulter', 'PR...","[[Robert, PROPN], [Boulter, PROPN], [is, AUX],...","[B-PERSON, Ī, O, O, O, O, O, O, O, O, O, O]","[[1, 2]]",[],"[robert, boulter, be, an, english, film, ,, te...","[Robert, Boulter, is, an, English, film, ,, te...","Robert Boulter is an English film , television..."
2,He had a guest @-@ starring role on the televi...,"{'toks': [['He', 'PRON'], ['had', 'VERB'], ['a...","[[He, PRON], [had, VERB], [a, DET], [guest, NO...","[O, O, O, O, O, O, O, O, O, B, Ī, O, O, O, O, O]","[[10, 11]]",[],"[he, have, a, guest, @-@, star, role, on, the,...","[He, had, a, guest, @-@, starring, role, on, t...",He had a guest @-@ starring role on the televi...
3,This was followed by a starring role in the pl...,"{'toks': [['This', 'PRON'], ['was', 'AUX'], ['...","[[This, PRON], [was, AUX], [followed, VERB], [...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-GROU...","[[14, 15], [24, 25]]",[],"[this, wa, follow, by, a, star, role, in, the,...","[This, was, followed, by, a, starring, role, i...",This was followed by a starring role in the pl...
4,He had a guest role in the television_series J...,"{'toks': [['He', 'PRON'], ['had', 'VERB'], ['a...","[[He, PRON], [had, VERB], [a, DET], [guest, AD...","[O, O, O, O, O, O, O, B-COMMUNICATION, Ī, O, B...","[[8, 9], [11, 12]]",[],"[he, have, a, guest, role, in, the, television...","[He, had, a, guest, role, in, the, television,...",He had a guest role in the television series J...
...,...,...,...,...,...,...,...,...,...
10260,Perhaps the greatest beneficiary of the film '...,"{'toks': [['Perhaps', 'ADV'], ['the', 'DET'], ...","[[Perhaps, ADV], [the, DET], [greatest, ADJ], ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",[],[],"[perhaps, the, greatest, beneficiary, of, the,...","[Perhaps, the, greatest, beneficiary, of, the,...",Perhaps the greatest beneficiary of the film '...
10261,Mosconi claimed in an interview at the time of...,"{'toks': [['Mosconi', 'PROPN'], ['claimed', 'V...","[[Mosconi, PROPN], [claimed, VERB], [in, ADP],...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[[18, 19], [21, 22, 23], [27, 28], [33, 34, 35]]",[],"[mosconi, claim, in, an, interview, at, the, t...","[Mosconi, claimed, in, an, interview, at, the,...",Mosconi claimed in an interview at the time of...
10262,Wanderone immediately adopted the Minnesota_Fa...,"{'toks': [['Wanderone', 'PROPN'], ['immediatel...","[[Wanderone, PROPN], [immediately, ADV], [adop...","[O, O, O, O, B-FOOD, Ī, O, O, O, O, O, O, O, O...","[[5, 6], [18, 19]]",[],"[wanderone, immediately, adopt, the, minnesota...","[Wanderone, immediately, adopted, the, Minneso...",Wanderone immediately adopted the Minnesota Fa...
10263,Author Walter_Tevis denied for the rest of his...,"{'toks': [['Author', 'NOUN'], ['Walter', 'PROP...","[[Author, NOUN], [Walter, PROPN], [Tevis, PROP...","[O, B-GROUP, Ī, O, O, O, O, O, O, O, O, O, O, ...","[[2, 3]]",[],"[author, walter, tevis, deny, for, the, rest, ...","[Author, Walter, Tevis, denied, for, the, rest...",Author Walter Tevis denied for the rest of his...


In [40]:
df['bpe_tokens'] = df.apply(lambda x: list(map(str, list(bpe_pipeline(x["sent"])))), axis=1)

In [42]:
df['token_map'] = df.apply(lambda x: tokenizations.get_alignments(x['tokens'], 
                                                                  x['bpe_tokens'])[1], axis=1)

In [43]:
df['token_map']

0                           [[0], [1], [2], [2], [2], [3]]
1        [[0], [1], [1], [1], [2], [3], [4], [5], [6], ...
2        [[0], [1], [2], [3], [4], [4], [4], [5], [6], ...
3        [[0], [1], [2], [3], [4], [5], [6], [7], [8], ...
4        [[0], [1], [2], [3], [4], [5], [6], [7], [8], ...
                               ...                        
10260    [[0], [1], [2], [3], [4], [5], [6], [7], [7], ...
10261    [[0], [0], [1], [2], [3], [4], [5], [6], [7], ...
10262    [[0], [0], [0], [1], [2], [3], [4], [5], [5], ...
10263    [[0], [1], [2], [2], [3], [4], [5], [6], [7], ...
10264    [[0], [1], [2], [3], [4], [5], [6], [7], [8], ...
Name: token_map, Length: 10265, dtype: object

In [55]:
aa = [[0], [1], [2], [2], [2], [3]]

In [57]:
def list_to_index_dict(input_list):
    index_dict = {}
    
    for index, sublist in enumerate(input_list):
        for item in sublist:
            if item in index_dict:
                index_dict[item].append(index)
            else:
                index_dict[item] = [index]
    
    return index_dict

In [59]:
df['token_map_dict'] = df['token_map'].apply(lambda x: list_to_index_dict(x))

In [60]:
df[['sent','tags', '_', '~', 'tokens','bpe_tokens', 'token_map', 'token_map_dict']]

,sent,tags,_,~,tokens,bpe_tokens,token_map,token_map_dict
0,= Robert Boulter =,"[O, B-GROUP, Ī, O]","[[2, 3]]",[],"[=, Robert, Boulter, =]","[=, ĠRobert, ĠB, oul, ter, Ġ=]","[[0], [1], [2], [2], [2], [3]]","{0: [0], 1: [1], 2: [2, 3, 4], 3: [5]}"
1,"Robert Boulter is an English film , television...","[B-PERSON, Ī, O, O, O, O, O, O, O, O, O, O]","[[1, 2]]",[],"[Robert, Boulter, is, an, English, film, ,, te...","[Robert, ĠB, oul, ter, Ġis, Ġan, ĠEnglish, Ġfi...","[[0], [1], [1], [1], [2], [3], [4], [5], [6], ...","{0: [0], 1: [1, 2, 3], 2: [4], 3: [5], 4: [6],..."
2,He had a guest @-@ starring role on the televi...,"[O, O, O, O, O, O, O, O, O, B, Ī, O, O, O, O, O]","[[10, 11]]",[],"[He, had, a, guest, @-@, starring, role, on, t...","[He, Ġhad, Ġa, Ġguest, Ġ@, -, @, Ġstarring, Ġr...","[[0], [1], [2], [3], [4], [4], [4], [5], [6], ...","{0: [0], 1: [1], 2: [2], 3: [3], 4: [4, 5, 6],..."
3,This was followed by a starring role in the pl...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, B-GROU...","[[14, 15], [24, 25]]",[],"[This, was, followed, by, a, starring, role, i...","[This, Ġwas, Ġfollowed, Ġby, Ġa, Ġstarring, Ġr...","[[0], [1], [2], [3], [4], [5], [6], [7], [8], ...","{0: [0], 1: [1], 2: [2], 3: [3], 4: [4], 5: [5..."
4,He had a guest role in the television series J...,"[O, O, O, O, O, O, O, B-COMMUNICATION, Ī, O, B...","[[8, 9], [11, 12]]",[],"[He, had, a, guest, role, in, the, television,...","[He, Ġhad, Ġa, Ġguest, Ġrole, Ġin, Ġthe, Ġtele...","[[0], [1], [2], [3], [4], [5], [6], [7], [8], ...","{0: [0], 1: [1], 2: [2], 3: [3], 4: [4], 5: [5..."
...,...,...,...,...,...,...,...,...
10260,Perhaps the greatest beneficiary of the film '...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",[],[],"[Perhaps, the, greatest, beneficiary, of, the,...","[Perhaps, Ġthe, Ġgreatest, Ġbeneficiary, Ġof, ...","[[0], [1], [2], [3], [4], [5], [6], [7], [7], ...","{0: [0], 1: [1], 2: [2], 3: [3], 4: [4], 5: [5..."
10261,Mosconi claimed in an interview at the time of...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[[18, 19], [21, 22, 23], [27, 28], [33, 34, 35]]",[],"[Mosconi, claimed, in, an, interview, at, the,...","[Mos, coni, Ġclaimed, Ġin, Ġan, Ġinterview, Ġa...","[[0], [0], [1], [2], [3], [4], [5], [6], [7], ...","{0: [0, 1], 1: [2], 2: [3], 3: [4], 4: [5], 5:..."
10262,Wanderone immediately adopted the Minnesota Fa...,"[O, O, O, O, B-FOOD, Ī, O, O, O, O, O, O, O, O...","[[5, 6], [18, 19]]",[],"[Wanderone, immediately, adopted, the, Minneso...","[W, ander, one, Ġimmediately, Ġadopted, Ġthe, ...","[[0], [0], [0], [1], [2], [3], [4], [5], [5], ...","{0: [0, 1, 2], 1: [3], 2: [4], 3: [5], 4: [6],..."
10263,Author Walter Tevis denied for the rest of his...,"[O, B-GROUP, Ī, O, O, O, O, O, O, O, O, O, O, ...","[[2, 3]]",[],"[Author, Walter, Tevis, denied, for, the, rest...","[Author, ĠWalter, ĠTe, vis, Ġdenied, Ġfor, Ġth...","[[0], [1], [2], [2], [3], [4], [5], [6], [7], ...","{0: [0], 1: [1], 2: [2, 3], 3: [4], 4: [5], 5:..."


In [78]:
def map_weak_mwes_together(x):
    
    mapped_mwes = []
    for j, mwe in enumerate(x['_']):
        mapped_mwes.append([])
        for index in mwe:
            
            if index-1 in x['token_map_dict']:
                for val in x['token_map_dict'][index-1]:
                    mapped_mwes[j].append(val+1)
                
                
    return mapped_mwes
                   
    
def map_strong_mwes_together(x):
    
    mapped_mwes = []
    for j, mwe in enumerate(x['~']):
        mapped_mwes.append([])
        for index in mwe:
            
            if index-1 in x['token_map_dict']:
                for val in x['token_map_dict'][index-1]:
                    mapped_mwes[j].append(val+1)
                
                
    return mapped_mwes
                    

In [82]:
df['weak_mwe'] = df.apply(lambda x: map_weak_mwes_together(x),axis=1)
df['strong_mwe'] = df.apply(lambda x: map_strong_mwes_together(x),axis=1)

In [83]:
df[['sent','tags', '_', 'weak_mwe', '~', 'strong_mwe', 'tokens','bpe_tokens', 'token_map', 'token_map_dict']]

,sent,tags,_,weak_mwe,~,strong_mwe,tokens,bpe_tokens,token_map,token_map_dict
0,= Robert Boulter =,"[O, B-GROUP, Ī, O]","[[2, 3]]","[[2, 3, 4, 5]]",[],[],"[=, Robert, Boulter, =]","[=, ĠRobert, ĠB, oul, ter, Ġ=]","[[0], [1], [2], [2], [2], [3]]","{0: [0], 1: [1], 2: [2, 3, 4], 3: [5]}"
1,"Robert Boulter is an English film , television...","[B-PERSON, Ī, O, O, O, O, O, O, O, O, O, O]","[[1, 2]]","[[1, 2, 3, 4]]",[],[],"[Robert, Boulter, is, an, English, film, ,, te...","[Robert, ĠB, oul, ter, Ġis, Ġan, ĠEnglish, Ġfi...","[[0], [1], [1], [1], [2], [3], [4], [5], [6], ...","{0: [0], 1: [1, 2, 3], 2: [4], 3: [5], 4: [6],..."
2,He had a guest @-@ starring role on the televi...,"[O, O, O, O, O, O, O, O, O, B, Ī, O, O, O, O, O]","[[10, 11]]","[[12, 13]]",[],[],"[He, had, a, guest, @-@, starring, role, on, t...","[He, Ġhad, Ġa, Ġguest, Ġ@, -, @, Ġstarring, Ġr...","[[0], [1], [2], [3], [4], [4], [4], [5], [6], ...","{0: [0], 1: [1], 2: [2], 3: [3], 4: [4, 5, 6],..."
3,This was followed by a starring role in the pl...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, B-GROU...","[[14, 15], [24, 25]]","[[15, 16], [25, 26]]",[],[],"[This, was, followed, by, a, starring, role, i...","[This, Ġwas, Ġfollowed, Ġby, Ġa, Ġstarring, Ġr...","[[0], [1], [2], [3], [4], [5], [6], [7], [8], ...","{0: [0], 1: [1], 2: [2], 3: [3], 4: [4], 5: [5..."
4,He had a guest role in the television series J...,"[O, O, O, O, O, O, O, B-COMMUNICATION, Ī, O, B...","[[8, 9], [11, 12]]","[[8, 9], [11, 12, 13]]",[],[],"[He, had, a, guest, role, in, the, television,...","[He, Ġhad, Ġa, Ġguest, Ġrole, Ġin, Ġthe, Ġtele...","[[0], [1], [2], [3], [4], [5], [6], [7], [8], ...","{0: [0], 1: [1], 2: [2], 3: [3], 4: [4], 5: [5..."
...,...,...,...,...,...,...,...,...,...,...
10260,Perhaps the greatest beneficiary of the film '...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",[],[],[],[],"[Perhaps, the, greatest, beneficiary, of, the,...","[Perhaps, Ġthe, Ġgreatest, Ġbeneficiary, Ġof, ...","[[0], [1], [2], [3], [4], [5], [6], [7], [7], ...","{0: [0], 1: [1], 2: [2], 3: [3], 4: [4], 5: [5..."
10261,Mosconi claimed in an interview at the time of...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[[18, 19], [21, 22, 23], [27, 28], [33, 34, 35]]","[[20, 21, 22], [24, 25, 26, 27], [31, 32], [37...",[],[],"[Mosconi, claimed, in, an, interview, at, the,...","[Mos, coni, Ġclaimed, Ġin, Ġan, Ġinterview, Ġa...","[[0], [0], [1], [2], [3], [4], [5], [6], [7], ...","{0: [0, 1], 1: [2], 2: [3], 3: [4], 4: [5], 5:..."
10262,Wanderone immediately adopted the Minnesota Fa...,"[O, O, O, O, B-FOOD, Ī, O, O, O, O, O, O, O, O...","[[5, 6], [18, 19]]","[[7, 8, 9], [23, 24]]",[],[],"[Wanderone, immediately, adopted, the, Minneso...","[W, ander, one, Ġimmediately, Ġadopted, Ġthe, ...","[[0], [0], [0], [1], [2], [3], [4], [5], [5], ...","{0: [0, 1, 2], 1: [3], 2: [4], 3: [5], 4: [6],..."
10263,Author Walter Tevis denied for the rest of his...,"[O, B-GROUP, Ī, O, O, O, O, O, O, O, O, O, O, ...","[[2, 3]]","[[2, 3, 4]]",[],[],"[Author, Walter, Tevis, denied, for, the, rest...","[Author, ĠWalter, ĠTe, vis, Ġdenied, Ġfor, Ġth...","[[0], [1], [2], [2], [3], [4], [5], [6], [7], ...","{0: [0], 1: [1], 2: [2, 3], 3: [4], 4: [5], 5:..."
